In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline

from innvestigator import InnvestigateModel
from inverter_util import Flatten
import sys
sys.path.append("..")

import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import argparse
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as Data
from VGG_16 import VGG16
import pickle

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
batch_size = 1
transform_train = transforms.Compose([
#             transforms.RandomCrop(32, padding=4),
#             transforms.RandomHorizontalFlip(),
            transforms.ToTensor()
    ])
transform_test = transforms.Compose([
        transforms.ToTensor()
])
train_data = torchvision.datasets.CIFAR10(
        root = './data/cifar-10',
        train = True,
        transform = transform_train,
        download = False
)
test_data = torchvision.datasets.CIFAR10(
        root = './data/cifar-10',
        train = False,
        transform = transform_test,
        download = False)
train_loader = Data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False)
test_loader = Data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

def pick_neurons_layer(relev, per=0.1): 
    rel = torch.sum(relev, [2, 3])
    print(rel)
    _, units = torch.topk(rel, round(per*rel.size(1)), dim=1)
    print(units.shape)
    return units.numpy()

In [3]:
seeds = [0]
dropouts = [True]


with torch.no_grad():
    for seed in seeds:
        for dropout in dropouts:
            model = VGG16(num_classes=10)
            if dropout:
                model_path = "../model_vgg_cifar/vgg_seed" + str(seed) + "_dropout_version2.pkl"
            else:
                model_path = "../model_vgg_cifar/vgg_seed" + str(seed) + "_nodropout_version2.pkl"
            checkpoint = torch.load(model_path)
            model.load_state_dict(checkpoint)
            model = model.cuda()
            model.eval()
                        # Convert to innvestigate model
            inn_model = InnvestigateModel(model, lrp_exponent=2,
                                          method="b-rule",
                                          beta=.5)
            for i, (data, target) in enumerate(train_loader):

                print("index:", i)

                data, target = data.cuda(), target.cuda()
                batch_size = int(data.size()[0])

                model_prediction, _ , true_relevance = inn_model.innvestigate(in_tensor=data)
                print(true_relevance[12].sum([0,2,3]))
                
                tmp = torch.sum(true_relevance[0].squeeze(), [1, 2])

                if i == 0:
                    relev = true_relevance
                else:
                    for l in range(len(relev)):
                        relev[l] = torch.cat((relev[l], true_relevance[l]), 0)
            print("done")
            print(len(relev))
            pers = [1.0]
            for per in pers:
                sample_neurons = {}
                for layer in range(len(relev)):
                    true_layer = 12-layer
                    r = relev[true_layer]
                    units = pick_neurons_layer(r, per)
                    for i in range(units.shape[0]):
                        if layer == 0:
                            sample_neurons[i] = []
                        sample_neurons[i].append(units[i])
                if dropout:
                    save_path = "./lrp_path_" + str(per) + "_seed" + str(seed) + "_train_version2.pkl"
                else:
                    save_path = "./lrp_path_" + str(per) + "_seed" + str(seed) + "_train_nodropout_version2.pkl"
                output = open(save_path, 'wb')
                pickle.dump(sample_neurons, output)  
            print("done")

index: 0
tensor([ 3.5003e-03,  0.0000e+00, -9.1019e-05,  3.1745e-02,  0.0000e+00,
         1.9547e-01,  1.0449e-01,  0.0000e+00,  0.0000e+00, -3.1143e-03,
         0.0000e+00,  3.5061e-03,  9.3393e-04,  1.5867e-01,  3.8372e-01,
         3.2677e-02,  2.4927e-01,  2.5888e-01,  1.4517e-03,  8.5548e-05,
         3.4265e-04,  5.7819e-02,  1.6104e-01,  1.0073e-02,  2.0068e-01,
         2.3496e-03,  1.3819e-02, -4.7889e-07,  1.1143e-01,  9.0164e-02,
         0.0000e+00,  0.0000e+00, -1.3625e-03,  0.0000e+00,  0.0000e+00,
         3.8446e-01,  8.2332e-02,  2.2673e-01,  9.9698e-01,  6.1585e-02,
         1.8725e-05,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
        -1.0512e-03,  3.3511e-01,  0.0000e+00,  1.8073e-01,  1.0075e-02,
         3.4594e-01, -1.2502e-04,  1.8421e-01,  1.7827e-02,  4.4349e-02,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.8113e-01,
         4.6854e-04,  0.0000e+00,  9.8518e-02,  0.0000e+00])
index: 1
tensor([-3.0430e-03,  0.0000e+00,  1.5986e-04

tensor([ 3.5095e-04,  0.0000e+00,  1.0775e-04,  8.2093e-02,  0.0000e+00,
         2.7202e-01,  1.0813e-01,  0.0000e+00,  0.0000e+00, -3.9389e-03,
         0.0000e+00,  4.2196e-03,  4.9652e-02,  1.8036e-01,  3.5202e-01,
         5.1637e-02,  4.3974e-01,  2.0347e-01,  3.0359e-03,  4.0067e-02,
         6.4197e-04,  5.0118e-02,  1.8570e-01,  9.6376e-03,  2.3514e-01,
         7.2905e-02,  4.3627e-03, -5.0438e-04,  7.6039e-02,  3.1971e-01,
         0.0000e+00,  0.0000e+00, -2.1681e-04,  0.0000e+00,  0.0000e+00,
         3.9959e-01,  1.1673e-01,  2.5678e-01,  1.2368e+00,  3.9759e-02,
         1.3123e-03,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
        -5.2654e-04,  3.7004e-01,  0.0000e+00,  2.5971e-01,  2.7883e-03,
         3.3588e-01, -1.3915e-03,  2.8513e-01,  4.2281e-03,  5.3274e-02,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  2.6269e-01,
         2.4126e-04,  0.0000e+00,  1.2195e-01,  0.0000e+00])
index: 10
tensor([-3.5182e-04,  0.0000e+00,  1.8281e-04,  1.599

KeyboardInterrupt: 

In [ ]:
seeds = [0]
dropouts = [True, False]


with torch.no_grad():
    for seed in seeds:
        for dropout in dropouts:
            model = VGG16(num_classes=10)
            if dropout:
                model_path = "../vgg_seed" + str(seed) + "_dropout_version2.pkl"
            else:
                model_path = "../vgg_seed" + str(seed) + "_nodropout_version2.pkl"
            checkpoint = torch.load(model_path)
            model.load_state_dict(checkpoint)
            model = model.cuda()
            model.eval()
                        # Convert to innvestigate model
            inn_model = InnvestigateModel(model, lrp_exponent=2,
                                          method="e-rule",
                                          beta=.5)
            for i, (data, target) in enumerate(test_loader):

                print("index:", i)

                data, target = data.cuda(), target.cuda()
                batch_size = int(data.size()[0])

                model_prediction, _ , true_relevance = inn_model.innvestigate(in_tensor=data)

                if i == 0:
                    relev = true_relevance
                else:
                    for l in range(len(relev)):
                        relev[l] = torch.cat((relev[l], true_relevance[l]), 0)

            print("done")
            print(len(relev))
            pers = [0.1, 1.0]
            for per in pers:
                sample_neurons = {}
                for layer in range(len(relev)):
                    true_layer = 12-layer
                    r = relev[true_layer]
                    units = pick_neurons_layer(r, per)
                    for i in range(units.shape[0]):
                        if layer == 0:
                            sample_neurons[i] = []
                        sample_neurons[i].append(units[i])
                if dropout:
                    save_path = "./lrp_path_" + str(per) + "_seed" + str(seed) + "_test_version2.pkl"
                else:
                    save_path = "./lrp_path_" + str(per) + "_seed" + str(seed) + "_test_nodropout_version2.pkl"
                output = open(save_path, 'wb')
                pickle.dump(sample_neurons, output)  
            print("done")

In [ ]:
model.eval()
total = 0
with torch.no_grad():
    for test_step, (val_x, val_y) in enumerate(test_loader):
        print("step:", test_step)
        val_x = val_x.cuda()
        val_y = val_y.cuda()
        val_output = model(val_x)
        _, val_pred_y = val_output.max(1)
        if test_step == 0:
            correct = val_pred_y.eq(val_y).sum().item()
        else:
            correct += val_pred_y.eq(val_y).sum().item()
        total += val_y.size(0)
result = float(correct) * 100.0 / float(total)
print(result)

In [11]:
 for i, (data, target) in enumerate(test_loader):
    
    print("index:", i)

    data, target = data.cuda(), target.cuda()
    batch_size = int(data.size()[0])

    model_prediction, _ , true_relevance = inn_model.innvestigate(in_tensor=data)
    
    if i == 0:
        relev = true_relevance
    else:
        for l in range(len(relev)):
            relev[l] = torch.cat((relev[l], true_relevance[l]), 0)

print("done")
print(len(relev))

index: 0
index: 1
index: 2
index: 3
done
13


In [13]:
pers = [0.01, 0.03, 0.05, 0.1]
for per in pers:
    sample_neurons = {}
    for layer in range(len(relev)):
        true_layer = 12-layer
        r = relev[true_layer]
        units = pick_neurons_layer(r, per)
        for i in range(units.shape[0]):
            if layer == 0:
                sample_neurons[i] = []
            sample_neurons[i].append(units[i])

    save_path = "./lrp_path_" + str(per) + "_seed" + str(seed) +  "_test.pkl"
    output = open(save_path, 'wb')
    pickle.dump(sample_neurons, output)  
print("done")

torch.Size([10000, 1])
torch.Size([10000, 1])
torch.Size([10000, 1])
torch.Size([10000, 1])
torch.Size([10000, 3])
torch.Size([10000, 3])
torch.Size([10000, 3])
torch.Size([10000, 5])
torch.Size([10000, 5])
torch.Size([10000, 5])
torch.Size([10000, 5])
torch.Size([10000, 5])
torch.Size([10000, 5])
torch.Size([10000, 2])
torch.Size([10000, 2])
torch.Size([10000, 4])
torch.Size([10000, 4])
torch.Size([10000, 8])
torch.Size([10000, 8])
torch.Size([10000, 8])
torch.Size([10000, 15])
torch.Size([10000, 15])
torch.Size([10000, 15])
torch.Size([10000, 15])
torch.Size([10000, 15])
torch.Size([10000, 15])
torch.Size([10000, 3])
torch.Size([10000, 3])
torch.Size([10000, 6])
torch.Size([10000, 6])
torch.Size([10000, 13])
torch.Size([10000, 13])
torch.Size([10000, 13])
torch.Size([10000, 26])
torch.Size([10000, 26])
torch.Size([10000, 26])
torch.Size([10000, 26])
torch.Size([10000, 26])
torch.Size([10000, 26])
torch.Size([10000, 6])
torch.Size([10000, 6])
torch.Size([10000, 13])
torch.Size([10000,

In [3]:
batch_size = 2560
attack = "pgd"
seed = 32
model = VGG16(num_classes=10)

model_path = "../vgg_seed" + str(seed) + "_dropout.pkl"

checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint)
model = model.cuda()
model.eval()

inn_model = InnvestigateModel(model, lrp_exponent=2,
                                          method="e-rule",
                                          beta=.5)

def read_data_label(data_path, label_path):
    with open(data_path, 'rb') as fr:
        test_data = pickle.load(fr)
        size = len(test_data)
    with open(label_path, 'rb') as fr:
        test_label = pickle.load(fr)
    return test_data, test_label, size

def pick_neurons_layer(relev, per=0.1): 
    rel = torch.sum(relev, [2, 3])
    _, units = torch.topk(rel, round(per*rel.size(1)), dim=1)
    print(units.shape)
    return units.numpy()

if attack == "pgd":
    test_data_path = "../data/adversarial_samples/Vanilla/pgd/test_adv(eps_0.031).pkl"
    test_label_path = "../data/adversarial_samples/Vanilla/pgd/test_label.pkl"
    test_data, test_label, size = read_data_label(test_data_path, test_label_path)
    dataset = Data.TensorDataset(test_data, test_label)
    
data_loader = Data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False)

for i, (data, target) in enumerate(data_loader):
    
    print("index:", i)

    data, target = data.cuda(), target.cuda()
    batch_size = int(data.size()[0])

    model_prediction, _ , true_relevance = inn_model.innvestigate(in_tensor=data)
    
    if i == 0:
        relev = true_relevance
    else:
        for l in range(len(relev)):
            relev[l] = torch.cat((relev[l], true_relevance[l]), 0)

print("done")
print(len(relev))

pers = [0.1, 1.0]
for per in pers:
    sample_neurons = {}
    for layer in range(len(relev)):
        true_layer = 12-layer
        r = relev[true_layer]
        units = pick_neurons_layer(r, per)
        for i in range(units.shape[0]):
            if layer == 0:
                sample_neurons[i] = []
            sample_neurons[i].append(units[i])
            
    save_path = "./lrp_path_" + str(per) + "_seed" + str(seed) + "_adv.pkl"
    
    output = open(save_path, 'wb')
    pickle.dump(sample_neurons, output)  
print("done")

index: 0
index: 1
index: 2
index: 3
done
13
torch.Size([10000, 6])
torch.Size([10000, 6])
torch.Size([10000, 13])
torch.Size([10000, 13])
torch.Size([10000, 26])
torch.Size([10000, 26])
torch.Size([10000, 26])
torch.Size([10000, 51])
torch.Size([10000, 51])
torch.Size([10000, 51])
torch.Size([10000, 51])
torch.Size([10000, 51])
torch.Size([10000, 51])
torch.Size([10000, 64])
torch.Size([10000, 64])
torch.Size([10000, 128])
torch.Size([10000, 128])
torch.Size([10000, 256])
torch.Size([10000, 256])
torch.Size([10000, 256])
torch.Size([10000, 512])
torch.Size([10000, 512])
torch.Size([10000, 512])
torch.Size([10000, 512])
torch.Size([10000, 512])
torch.Size([10000, 512])
done
